In [1]:
imagePath = "/work/marcus_mueller/aeolus/plots_neu2/"
from plot import *
import os
from multiprocessing import Process, Queue
from sklearn.neighbors import NearestNeighbors
import pandas as pd
os.putenv('CODA_DEFINITION', '/home/mmueller/hiwi/aeolus/')

import coda
from numpy import vstack, zeros
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("Agg")
import pyproj
import geopandas as gpd
from shapely.ops import nearest_points
from shapely.geometry import LineString
import sys, os
import xarray as xr

from radarlidar_analysis.RadarLidarWindSpeed import RadarLidarWindSpeed
from datetime import datetime, timedelta
import tarfile
import math

import seaborn as sns
sns.set_theme()

orbit = '3082'
path = '/work/marcus_mueller/aeolus/'

fileList =  os.listdir(path+orbit+"/")
tasks = []
tasks.append(fileList[::4])
tasks.append(fileList[1::4])
tasks.append(fileList[2::4])
tasks.append(fileList[3::4])

dt = getMeasurementTime(fileList[0])

In [15]:
filename = "meteogram.iglo.h."+dt.strftime("%Y%m%d")+"00.nc"
path = "/data/mod/icon_op/iglo/site/"+dt.strftime("%Y/%m/")+filename
ds = xr.open_dataset(path)
nStation = ds.station_name.values.tolist().index(b'Juelich')
nU = ds.var_name.values.tolist().index(b'U')
nV = ds.var_name.values.tolist().index(b'V')
height = ds.sel(nstations=nStation, nvars = [nU],nsfcvars=[],time=4)['heights'].values.flatten().tolist()
u = ds.sel(nstations=nStation, nvars = [nU],nsfcvars=[],time=4)['values'].values.flatten().tolist()
v = ds.sel(nstations=nStation, nvars = [nV],nsfcvars=[],time=4)['values'].values.flatten().tolist()
